In [1]:
data_path_list = ["/Volumes/Extreme SSD/P02_MPAOPTO_LP/e04_ephys-contrapag-stim/v01/M22/2024-04-23_10-39-40"]
run_barcodeSync = False
run_preprocessing = True # run preprocessing and spikesorting
callKSfromSI = False  # this remains insanely slow.

In [5]:
%matplotlib widget
from matplotlib import pyplot as plt
import spikeinterface.extractors as se
import spikeinterface.widgets as sw
import spikeinterface.preprocessing as st

from spikeinterface import get_noise_levels, aggregate_channels
from pathlib import Path
import os

In [ ]:
streams = ['Record Node 103#NI-DAQmx-102.USB-6212 (BNC)', 'Record Node 103#Neuropix-PXI-100.ProbeA-AP', 
           'Record Node 103#Neuropix-PXI-100.ProbeA-LFP', 'Record Node 103#Neuropix-PXI-100.ProbeB-AP', 
           'Record Node 103#Neuropix-PXI-100.ProbeB-LFP']
bnc_data = se.read_openephys(data_path, stream_name=streams[0])
recording = se.read_openephys(data_path, stream_name=streams[1])


fs = recording.get_sampling_frequency()

noise_levels_microV = get_noise_levels(recording, return_scaled=True)

# preprocessing steps
recording_lsb = st.correct_lsb(recording, verbose=1)
[bad_channel_ids, channel_labels] = st.detect_bad_channels(recording=recording)  
recording = recording.remove_channels(remove_channel_ids=bad_channel_ids)  # could be interpolated instead, but why?

recording_bandpass = st.bandpass_filter(recording, freq_min=300, freq_max=6000)
recording_ps = st.phase_shift(recording_bandpass) #lazy

grouped_recordings = recording.split_by(property='group')

# Spatial filtering function and parameters to use on either the recording or the recording split in groups:
spatial_filter_func = st.highpass_spatial_filter
spatial_filter_params = {}

# split in groups and apply spatial filtering, then reaggregate. KS4 can now handle multiple shanks
if not all(recording_ps.get_property("group") == 0):
    grouped_recordings = recording_ps.split_by(property='group')
    recgrouplist_hpsf = [spatial_filter_func(recording=grouped_recordings[k], **spatial_filter_params) for k in grouped_recordings.keys()]  # cmr is slightly faster. results are similar
    recording_hpsf = aggregate_channels(recgrouplist_hpsf)
else:
    recording_hpsf = spatial_filter_func(recording=recording_ps, **spatial_filter_params)

In [31]:
w_ts = sw.plot_traces({"traces": recording, 
                       "bandpass": recording_bandpass,
                       "recording_hpsf": recording_hpsf}, 
                       backend="ipywidgets",time_range=[0, 0.05], width_cm=30, height_cm=10)
# w_ts.ax.plot([0, 5], [200, 200])

AppLayout(children=(TimeSlider(children=(Dropdown(description='segment', options=(0,), value=0), Button(icon='…

: 

In [8]:

# split in groups and apply spatial filtering, then reaggregate. KS4 can now handle multiple shanks
grouped_recordings = recording.split_by(property='group')
recgrouplist_hpsf = [st.highpass_spatial_filter(recording=grouped_recordings[k]) for k in grouped_recordings.keys()]  # cmr is slightly faster. results are similar
recording_hpsf = si.aggregate_channels(recgrouplist_hpsf)
print(recording_hpsf)

Signature:
st.correct_lsb(
    recording,
    num_chunks_per_segment=20,
    chunk_size=10000,
    seed=None,
    verbose=False,
)
Docstring:
Estimates the LSB of the recording and divide traces by LSB
to ensure LSB = 1. Medians are also subtracted to avoid rounding errors.

Parameters
----------
recording : RecordingExtractor
    The recording extractor to be LSB-corrected.
num_chunks_per_segment : int, default: 20
    Number of chunks per segment for random chunk
chunk_size : int, default: 10000
    Size of a chunk in number for random chunk
seed : int or None, default: None
    Random seed for random chunk
verbose : bool, default: False
    If True, estimate LSB value is printed

Returns
-------
correct_lsb_recording : ScaleRecording
    The recording extractor with corrected LSB
File:      ~/code/spikeinterface/src/spikeinterface/preprocessing/correct_lsb.py
Type:      function

AppLayout(children=(TimeSlider(children=(Dropdown(description='segment', options=(0,), value=0), Button(icon='…